In [ ]:
# self.bn = nn.BatchNorm2d(64)
# self.mp = nn.MaxPool2d(kernel_size = (12,22),stride =(1,1))
# self.dp = nn.Dropout(p=0.2)
# self.sm = nn.Softmax(dim = 0)
# self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
# self.relu1 = nn.ReLU()
# self.maxpool1 = nn.MaxPool2d(kernel_size=4)
# self.fc1 = nn.Linear(576, 10)

Step 1 : Build your model

In [12]:
import torch.nn as nn
class My_Model(nn.Module):
    def __init__(self):
        super(My_Model, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=4)
        self.fc1 = nn.Linear(576, 10)
    def forward(self, x):
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

Step 2 : Load input and label ,convert them to torch variable

In [13]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets
import random


class CustomDataset(Dataset):
    def __init__(self, csv_path):

        # Transforms
        self.to_tensor = transforms.ToTensor()
        
        # Read the csv file
        self.data_info = pd.read_csv(csv_path, header=None)

        # First column is image paths
        self.path_arr = np.asarray(self.data_info.iloc[:, 0])
        
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        
        # Calculate sample counts
        self.data_len = len(self.data_info.index)

        
    def __getitem__(self, index):
        

        path = self.path_arr[index]


#         x = np.load(path)['arr_0']
        x = Image.open(path)

        #start preprocess data

        #end preprocess data
        
#         x = torch.from_numpy(x)
        x = self.to_tensor(x)

        label = int(self.label_arr[index])
        
        return (x.float(), label)

    def __len__(self):
        return self.data_len


custom_data = CustomDataset('./train.csv')


dataset_loader = torch.utils.data.DataLoader(dataset=custom_data,
                                                batch_size=10,
                                                shuffle=True)

model = My_Model()
criterion = nn.CrossEntropyLoss()
optimizer =  torch.optim.Adam(model.parameters(), lr = 0.001)
epoch = 1

# Clear gradients
# optimizer.zero_grad()

for ep in range(epoch):
    print('ep',ep)
    for i, (x, labels) in enumerate(dataset_loader):
        x = Variable(x)
        labels = Variable(labels)
        # Forward pass
        outputs = model(x)
        
        # Calculate loss 
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()

    _,idx = torch.max(outputs, 1)
    print(idx.data.numpy(),labels.data.numpy())
    

('ep', 0)
(array([3, 9, 3, 3, 3, 3, 3, 2, 3]), array([3, 9, 0, 6, 5, 0, 9, 4, 1]))


Step 3: Training

In [9]:
epoch = 10

for ep in range(epoch):

    for i, (xs, labels) in enumerate(dataset_loader):
        xs = Variable(xs)
        labels = Variable(labels)
        # Forward pass
        outputs = model(xs)
        # Calculate loss
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        # Backward pass
        loss.backward()
        # Update weights
        optimizer.step()

        if i == 0: 
            _,idx = torch.max(outputs, 1)
            
            if ep%10 ==0:
                print('ep',ep)
                print(loss)
                print(idx.data.numpy(),labels.data.numpy())


('ep', 0)
Variable containing:
1.00000e-02 *
  1.3286
[torch.FloatTensor of size 1]

(array([8, 4, 4, 0, 9, 3, 5, 6, 9, 5]), array([8, 4, 4, 0, 9, 3, 5, 6, 9, 5]))
